<a href="https://colab.research.google.com/github/loktissimo/Raketa/blob/main/Petrovax_report_cutter_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  @title Загрузи файл-реестр Petorvax с актами

import pandas as pd
import io

from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())

print(f"\n✔️ Готово. Файл - {filename[0]}")

Saving Petrovax_report_16-30.09.2024 (2).xlsx to Petrovax_report_16-30.09.2024 (2).xlsx

✔️ Готово. Файл - Petrovax_report_16-30.09.2024 (2).xlsx


In [ ]:
#  @title Хочешь распилить файл на кусочки - НАЖИМАЙ

import pandas as pd
import numpy as np


''' Загружаем данные из .xls в DataFrame '''
df = pd.read_excel(io.BytesIO(uploaded[filename[0]]), header=[0])

cfo_name_list = df['ЦФО'].unique()

''' Создаем папку для файлов '''
!mkdir report

for cfo_name in cfo_name_list:
    tickets_df = pd.DataFrame(columns=df.columns.values.tolist())
    services_df = pd.DataFrame(columns=df.columns.values.tolist())

    ''' Идем по строкам исходного файла '''
    for index, row in df.iterrows():
        if str(cfo_name) == str(row.iloc[9]) and str(row.iloc[15]) in ['Авиабилет', 'ЖД билет']:
            tickets_df.loc[len(tickets_df.index)] = df.iloc[index]
            cfo_code = str(row.iloc[10])
        elif str(cfo_name) == str(row.iloc[9]) and str(row.iloc[15]) in ['Сборы', 'Проживание', 'Трансфер', 'Допуслуга авиа', 'Штраф']:
            services_df.loc[len(services_df.index)] = df.iloc[index]
            cfo_code = str(row.iloc[10])


    ''' Добавление сумм в 3х столбцах '''
    tickets_df.loc['', 'Стоимость услуги без НДС'] = tickets_df['Стоимость услуги без НДС'].sum()
    services_df.loc['', 'Стоимость услуги без НДС'] = services_df['Стоимость услуги без НДС'].sum()
    tickets_df.loc['', 'Сумма НДС'] = tickets_df['Сумма НДС'].sum()
    services_df.loc['', 'Сумма НДС'] = services_df['Сумма НДС'].sum()
    tickets_df.loc['', 'Стоимость услуги с НДС'] = tickets_df['Стоимость услуги с НДС'].sum()
    services_df.loc['', 'Стоимость услуги с НДС'] = services_df['Стоимость услуги с НДС'].sum()


    ''' Заполнение столбца с нумерацией строк '''
    tickets_df['Номер п/п'] = range(1, len(tickets_df) + 1)
    tickets_df.loc[tickets_df.index[-1], 'Номер п/п'] = None
    services_df['Номер п/п'] = range(1, len(services_df) + 1)
    services_df.loc[services_df.index[-1], 'Номер п/п'] = None


    ''' Дополнительные заголовки конечных файлов '''
    header1 = ['Реестр корпоративного клиента №  за период 01-15.06.2024']
    header2 = ['Реестр билетов', '', cfo_code, cfo_name, 'НПО-340-2023 от 01.05.2023']

    ''' Не создаем файлы без билетов (*Билеты_nan_...) '''
    # print(f'{tickets_df.iat[0, 4] = }; {type(tickets_df.iat[0, 4])} \
    #     \n {services_df.iat[0, 4] = }; {type(services_df.iat[0, 4])}')
    # if np.isnan(tickets_df.iat[0, 4]) or np.isnan(services_df.iat[0, 4]):
    #     continue

    tickets_filename = f"report/Билеты_{tickets_df.iat[0, 4]}_{cfo_name}.xlsx"
    services_filename = f"report/Услуги_{services_df.iat[0, 4]}_{cfo_name}.xlsx"

    with pd.ExcelWriter(tickets_filename) as writer:
        writer.book.create_sheet('Sheet1')
        writer.sheets['Sheet1'].append(header1)
        writer.sheets['Sheet1'].append(header2)
        tickets_df.infer_objects(copy=False).to_excel(writer, sheet_name='Sheet1', index=False, startrow=2)

    with pd.ExcelWriter(services_filename) as writer:
        writer.book.create_sheet('Sheet1')
        writer.sheets['Sheet1'].append(header1)
        writer.sheets['Sheet1'].append(header2)
        services_df.infer_objects(copy=False).to_excel(writer, sheet_name='Sheet1', index=False, startrow=2)

print(f"\n✔️ Готово.")



✔️ Готово.


In [ ]:
#  @title Скачать все файлы архивом (скачается само, надо подождать)

from datetime import datetime, timezone
from google.colab import files
# from google.colab import drive

DATE_FILE = datetime.now().strftime("%H-%M--%d.%m.%Y")

# drive.mount('/content/')
file_name="Petrovax_report" + DATE_FILE + ".zip"
!zip -r "$file_name" ./report/

files.download(file_name)

!rm -r ./report

print(f"\n✔️ Готово.")
